# Lecture 3: Spark Streaming

Spark streaming is an extension of the Spark API that enables scalabe stream processing. The stream of data can be ingested from many data sources such as *Kafka*, *s3* or *TCP socket*. The Spark API allows to process data via high-level functions such as *map* and *reduce*. As we are going to se, it is also possible to use dataframe operations. 

Processed data can be exported to an external database and used to make live dashboards or offline analysis. 

Spark streaming works by dividing the input data into micro-batches that can be treated as static dataset. In Spark this is called *discretized stream* (*DStream*). The DStream is represented using RDDs.

![DStream](imgs/lecture3/DStream.png)

A transformation of the DStream, i.e. `Dstream.map` will act independently on each batch. For example, in the image below, we can filter the original RDD to remove some data and produce a new stream. 

![DStream_filter](./imgs/lecture3/Dstream_filter.png)

In this lecture we will see how to setup a simple stream using as source a socket.

## Setup

In [2]:
# set this variable with one of the following values
# -> 'local'
# -> 'docker_container'
# -> 'docker_cluster'
CLUSTER_TYPE ='local'

In [3]:
# set env variable
%env CLUSTER_TYPE $CLUSTER_TYPE

env: CLUSTER_TYPE=local


## Start the cluster 

Environment variables need to be set only in the case of a local cluster

In [40]:
if CLUSTER_TYPE=='local':
    import findspark
    findspark.init('/home/lorenzo/spark-3.1.1-bin-hadoop3.2')

In [41]:
%%script bash --no-raise-error

if [[ "$CLUSTER_TYPE" != "docker_cluster" ]]; then
    echo "Launching master and worker"
    
    # start master 
    $SPARK_HOME/sbin/start-master.sh --host localhost \
        --port 7077 --webui-port 8080
    
    # start worker
    $SPARK_HOME/sbin/start-worker.sh spark://localhost:7077 \
        --cores 2 --memory 2g
fi

Launching master and worker
org.apache.spark.deploy.master.Master running as process 8638.  Stop it first.
org.apache.spark.deploy.worker.Worker running as process 8722.  Stop it first.


## Create the Spark session

Later on we will explain what is the role of [Apache Arrow](), but first we need to install it and create the spark session with it.

In [42]:
from pyspark.sql import SparkSession

if CLUSTER_TYPE in ['local', 'docker_container']:
    
    spark = SparkSession.builder \
        .master("spark://localhost:7077")\
        .appName("Spark streaming application")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .getOrCreate()

elif CLUSTER_TYPE == 'docker_cluster':
    
    # use the provided master
    spark = SparkSession.builder \
        .master("spark://spark-master:7077")\
        .appName("Spark streaming application")\
        .config("spark.executor.memory", "512m")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .getOrCreate()

In [43]:
sc = spark.sparkContext
sc

<SparkContext master=spark://localhost:7077 appName=Spark streaming application>

## Streaming context

The first step of a spark streaming application is the creation of a StreamingContext. The context can be initialized using `StreamingContext(SparkContext, batch_interval`). There could be at most one StreamingContext for each spark application. The processing start when `streamingContext.start()` is called and it can be stopped with `streamingContext.stop()`. If it is stoped without passing `stop(stopSparkContext=False)` the sparkContext is also stopped.

In [86]:
# create streaming context
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc, 2)

For this example spark will read data from a TCP socked. Data are generated by a simple python program that can be found in `utils/producer.py`. The producer will write data on port `5555` of `localhost`. 

The dataset consists of fake credit card transactions.

In [87]:
# read stream from socket
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)


The first thing we need to to is load the json describing each transaction.

In [88]:
# load json files
import json

json_stream = socket_stream.map(lambda msg: json.loads(msg))

It is possible to print some elements of each batch with `pprint()`. This can be used to explore the RDDs.

In [14]:
# show elemnts
json_stream.pprint()

Start the computations with `ssc.start()` and stop with `ssc.stop(stopSparkContext=False)`. Remember that after this last instruction the streaming context must be defined again.

In [36]:
# start stream
ssc.start()

IllegalArgumentException: requirement failed: No output operations registered, so nothing to execute

In [47]:
# stop stream
ssc.stop(stopSparkContext=False)

Before running the following cells, create again the streaming context and run the cells where the input data source is defined. Remember to skip `pprint` otherwise this operation will be appended to the DAG. 

We may want to convert each batch into a Spark DataFrame to have higher level API. To do that, let's first convert numeric features of the json into python floats and integers. The dictionary can then be used to create a `Row`for each transaction.

In [89]:
from pyspark.sql import Row

def create_row_rdd(t):
    # set data types and create
    # dataframe rows
    
    t['amount'] = float(t['amount'])
    t['delta_t'] = float(t['delta_t'])
    t['flag'] = int(t['flag'])
    
    return Row(**t)


row_stream = json_stream.map(create_row_rdd)

The method `DStream.foreachRDD` can be used to apply custom transformations to each batch of data. In our case we are insterested in converting each batch of data into a dataframe and perform operations like finding the number of transactions for each user. Transacion where the user performed more than one transaction, with the flag equal to one,  inside the microbatch and can be flagged as fraudulent.

In [90]:
spark.conf.set('spark.sql.shuffle.partitions', 3)

In [91]:
from pyspark.sql.functions import concat, col, lit, countDistinct

def process_batch(rdd):
    # convert to df
    df = rdd.toDF(
    
        #if rdd empty operation will fail so I must write something inside. If no row I cannot 
        #infer a schema
        schema='name string, surname string, amount float, delta_t float, flag int'
    
    )
    
    # find number of transactions for each user when flag = 1 
    num_transactions = (
        df
        .where(col('flag')==1)
        #in reality I have an ID for each name surname
        .withColumn('id', concat(col('name'), col('surname')))
        .groupBy('id')
        .count()
    )
    
    # find suspicious transactions
    sus_transactions = (
        num_transactions.where(col('count') > 1).withColumn('fraud', lit(1))
        .select(col('id'), col('fraud'))
    )
    sus_transactions.show(5)

    
    # send allarm
    
    

row_stream.foreachRDD(process_batch)

In [92]:
ssc.start()

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---------+-----+
|       id|fraud|
+---------+-----+
|AndySmith|    1|
+---------+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+-----------+-----+
|         id|fraud|
+-----------+-----+
|JohnJohnson|    1|
+-----------+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|


In [93]:
# stop streaming context

ssc.stop(stopSparkContext=False)

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

